## Это наш лицевой файл, к которому будем подключать свои скрипты. Тут будут основные визуальные выводы. Логика вычислений и обработки будет вынесена в отдельные файлы .py


In [2]:
import sys
import os

cur_dir = os.getcwd()
dataset_path = os.path.join(cur_dir, "../Dataset")
script_path = os.path.join(cur_dir, "../Scripts")
wavelets_path = os.path.join(cur_dir, "../Wavelet")
print(wavelets_path)
sys.path.append(dataset_path)
sys.path.append(script_path)
sys.path.append(wavelets_path)

d:\УЧЕБА\Remote monitoring\Neural network\../Wavelet


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from keras.optimizers import SGD

In [4]:
from importlib import reload
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [5]:
import ECGsignal as ecg

reload(ecg)

ecg.init()

d:\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [6]:
import CWT as wt

In [7]:
reload(wt)
wt.init(len_sig=1500)

In [8]:

test_ecg = ecg.signal(number_labels={"VA" : 0, "AF" : 300, "SR" : 300})

wt.cwt_transform_df(test_ecg, 0, 3, wt.wavelet()[3])


,data,label,CWT
0,"[[-0.025, -0.025, -0.025, -0.025, -0.025, -0.0...",AF,"[[[0.6175382237409982, 0.4560149675525502, 0.3..."
1,"[[-0.375, -0.375, -0.375, -0.375, -0.375, -0.3...",AF,"[[[1.2365761211891104, 1.0204712539461895, 0.8..."
2,"[[-0.175, -0.175, -0.175, -0.175, -0.176, -0.1...",AF,"[[[1.5871337329788624, 1.1757084536588893, 0.7..."
3,"[[-0.12, -0.12, -0.121, -0.119, -0.123, -0.12,...",AF,"[[[1.4510746452505487, 1.083377088451957, 0.75..."
4,"[[-0.005, -0.005, -0.005, -0.005, -0.005, -0.0...",AF,"[[[1.914440571286241, 1.408174157425532, 0.980..."
...,...,...,...
595,"[[0.115, 0.115, 0.115, 0.115, 0.115, 0.115, 0....",SR,"[[[2.081467820245389, 1.5076489139773284, 0.91..."
596,"[[0.015, 0.015, 0.015, 0.015, 0.014, 0.015, 0....",SR,"[[[0.8531937295150357, 0.6457379626800893, 0.5..."
597,"[[-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0...",SR,"[[[0.8486110415149096, 0.6035817399328598, 0.3..."
598,"[[-0.12, -0.12, -0.12, -0.12, -0.121, -0.12, -...",SR,"[[[1.6971356335708265, 1.2391536465595694, 0.8..."


In [8]:
test_ecg["CWT"][1].shape

(2, 108, 1500)

In [9]:
test_ecg = test_ecg.replace({"label" : {"SR":0, "VA":1, "AF":2}})

In [10]:
X_train, X_test, y_train, y_test \
    = train_test_split(np.array(test_ecg["CWT"]), np.array(test_ecg["label"]), test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val \
    = train_test_split(np.array(X_train), np.array(y_train), test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

# train 80%, test 20%, val 20%

In [11]:
X_train = np.stack(X_train, axis=0)
X_val = np.stack(X_val, axis=0)
X_test = np.stack(X_test, axis=0)

In [12]:
# Преобразование меток в one-hot кодировку
# y_train = to_categorical(y_train, num_classes=3).numpy()
# y_val = to_categorical(y_val, num_classes=3)
# y_test = to_categorical(y_test, num_classes=3)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[2], X_train.shape[3], X_train.shape[1]))
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_val = X_val.reshape((X_val.shape[0], X_val.shape[2], X_val.shape[3], X_val.shape[1]))
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)

X_test = X_test.reshape((X_test.shape[0], X_test.shape[2], X_test.shape[3], X_test.shape[1]))
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)

y_train = tf.one_hot(y_train, depth=2).numpy()
y_val = tf.one_hot(y_val, depth=2).numpy()
y_test = tf.one_hot(y_test, depth=2).numpy()

In [13]:
X_val.shape

TensorShape([120, 108, 1500, 2])

In [ ]:


# Предположим, что X_train, y_train, X_val, y_val, X_test, y_test - это ваши данные
spec_height = test_ecg["CWT"][0].shape[1]
spec_width = test_ecg["CWT"][0].shape[2]
channels = 2  # для двух каналов в спектрограмме
conv_shape = 5
opt = SGD(learning_rate = 0.0001)

# Создание модели
model = models.Sequential()
model.add(layers.Conv2D(32, (conv_shape, conv_shape), activation='relu', input_shape=(spec_height, spec_width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# Компиляция модели
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=15, validation_data=(X_val, y_val))

# Оценка производительности на тестовом наборе
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


Epoch 1/15
1/1 [==============================] - 2s 2s/step - loss: 1.0777 - accuracy: 0.3889 - val_loss: 1.1326 - val_accuracy: 0.0000e+00
Epoch 2/15
1/1 [==============================] - 1s 1s/step - loss: 1.1517 - accuracy: 0.2222 - val_loss: 1.1309 - val_accuracy: 0.0000e+00
Epoch 3/15
1/1 [==============================] - 1s 1s/step - loss: 1.0313 - accuracy: 0.4444 - val_loss: 1.1241 - val_accuracy: 0.0000e+00
Epoch 4/15
1/1 [==============================] - 1s 1s/step - loss: 1.1061 - accuracy: 0.3889 - val_loss: 1.1262 - val_accuracy: 0.0000e+00
Epoch 5/15
1/1 [==============================] - 1s 1s/step - loss: 1.1011 - accuracy: 0.2778 - val_loss: 1.1253 - val_accuracy: 0.0000e+00
Epoch 6/15
1/1 [==============================] - 1s 1s/step - loss: 0.9790 - accuracy: 0.5000 - val_loss: 1.1238 - val_accuracy: 0.1667
Epoch 7/15
1/1 [==============================] - 1s 1s/step - loss: 1.0597 - accuracy: 0.3889 - val_loss: 1.1245 - val_accuracy: 0.1667
Epoch 8/15
1/1 [=====

# Выше результаты для вейвлета морле с выборкой 300


In [121]:

# Предположим, что X_train, y_train, X_val, y_val, X_test, y_test - это ваши данные
spec_height = test_ecg["CWT"][0].shape[1]
spec_width = test_ecg["CWT"][0].shape[2]
channels = 2  # для двух каналов в спектрограмме
conv_shape = 5
opt = SGD(learning_rate = 0.0001)

# Создание модели
model = models.Sequential()
model.add(layers.Conv2D(32, (conv_shape, conv_shape), activation='relu', input_shape=(spec_height, spec_width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# Компиляция модели
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val))

# Оценка производительности на тестовом наборе
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


Epoch 1/25
17/17 [==============================] - 32s 2s/step - loss: 1.1115 - accuracy: 0.3981 - val_loss: 1.0417 - val_accuracy: 0.4611
Epoch 2/25
17/17 [==============================] - 34s 2s/step - loss: 1.0380 - accuracy: 0.4519 - val_loss: 0.9973 - val_accuracy: 0.5056
Epoch 3/25
17/17 [==============================] - 32s 2s/step - loss: 0.9787 - accuracy: 0.5241 - val_loss: 0.9723 - val_accuracy: 0.5056
Epoch 4/25
17/17 [==============================] - 32s 2s/step - loss: 0.9631 - accuracy: 0.5259 - val_loss: 0.9471 - val_accuracy: 0.5167
Epoch 5/25
17/17 [==============================] - 46s 3s/step - loss: 0.9270 - accuracy: 0.5444 - val_loss: 0.9278 - val_accuracy: 0.5611
Epoch 6/25
17/17 [==============================] - 44s 3s/step - loss: 0.8948 - accuracy: 0.5611 - val_loss: 0.9095 - val_accuracy: 0.5889
Epoch 7/25
17/17 [==============================] - 43s 3s/step - loss: 0.8836 - accuracy: 0.6111 - val_loss: 0.9026 - val_accuracy: 0.5667
Epoch 8/25
17/17 [==

# Выше результат для выборки в 900 для вейвлета мексики

In [12]:
# Предположим, что X_train, y_train, X_val, y_val, X_test, y_test - это ваши данные
spec_height = test_ecg["CWT"][0].shape[1]
spec_width = test_ecg["CWT"][0].shape[2]
channels = 2  # для двух каналов в спектрограмме
conv_shape = 5
opt = SGD(learning_rate = 0.0001)

# Создание модели
model = models.Sequential()
model.add(layers.Conv2D(32, (conv_shape, conv_shape), activation='relu', input_shape=(spec_height, spec_width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# Компиляция модели
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val))

# Оценка производительности на тестовом наборе
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)



Epoch 1/25


17/17 [==============================] - 51s 3s/step - loss: 1.1430 - accuracy: 0.3722 - val_loss: 1.0552 - val_accuracy: 0.5000
Epoch 2/25
17/17 [==============================] - 45s 3s/step - loss: 1.0467 - accuracy: 0.4519 - val_loss: 1.0098 - val_accuracy: 0.4778
Epoch 3/25
17/17 [==============================] - 43s 3s/step - loss: 0.9846 - accuracy: 0.5167 - val_loss: 0.9773 - val_accuracy: 0.5000
Epoch 4/25
17/17 [==============================] - 44s 3s/step - loss: 0.9715 - accuracy: 0.5241 - val_loss: 0.9563 - val_accuracy: 0.5111
Epoch 5/25
17/17 [==============================] - 43s 3s/step - loss: 0.9207 - accuracy: 0.5648 - val_loss: 0.9336 - val_accuracy: 0.5222
Epoch 6/25
17/17 [==============================] - 42s 2s/step - loss: 0.8944 - accuracy: 0.5926 - val_loss: 0.9095 - val_accuracy: 0.5833
Epoch 7/25
17/17 [==============================] - 40s 2s/step - loss: 0.8817 - accuracy: 0.6037 - val_loss: 0.9114 - val_accuracy: 0.5500
Epoch 8/25
17/17

In [13]:
# Предположим, что X_train, y_train, X_val, y_val, X_test, y_test - это ваши данные
spec_height = test_ecg["CWT"][0].shape[1]
spec_width = test_ecg["CWT"][0].shape[2]
channels = 2  # для двух каналов в спектрограмме
conv_shape_1 = 7
conv_shape_2 = 7
opt = SGD(learning_rate = 0.0001)

# Создание модели
model = models.Sequential()
model.add(layers.Conv2D(32, (conv_shape_1, conv_shape_2), activation='relu', input_shape=(spec_height, spec_width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (conv_shape_1, conv_shape_2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (conv_shape_1, conv_shape_2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# Компиляция модели
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val))

# Оценка производительности на тестовом наборе
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)



Epoch 1/25


6/6 [==============================] - 60s 9s/step - loss: 1.1301 - accuracy: 0.3444 - val_loss: 1.0951 - val_accuracy: 0.3500
Epoch 2/25
6/6 [==============================] - 49s 8s/step - loss: 1.0857 - accuracy: 0.3722 - val_loss: 1.0695 - val_accuracy: 0.4167
Epoch 3/25
6/6 [==============================] - 50s 8s/step - loss: 1.0982 - accuracy: 0.3833 - val_loss: 1.0685 - val_accuracy: 0.4167
Epoch 4/25
6/6 [==============================] - 45s 7s/step - loss: 1.0304 - accuracy: 0.4167 - val_loss: 1.0464 - val_accuracy: 0.4667
Epoch 5/25
6/6 [==============================] - 40s 7s/step - loss: 1.0140 - accuracy: 0.5000 - val_loss: 1.0431 - val_accuracy: 0.4167
Epoch 6/25
6/6 [==============================] - 40s 7s/step - loss: 1.0157 - accuracy: 0.4722 - val_loss: 1.0230 - val_accuracy: 0.5667
Epoch 7/25
6/6 [==============================] - 40s 7s/step - loss: 0.9745 - accuracy: 0.5167 - val_loss: 1.0282 - val_accuracy: 0.4500
Epoch 8/25
6/6 [==============

In [13]:
# saving the dataframe
test_ecg.to_csv('cwts_300.csv')

In [21]:
# Предположим, что X_train, y_train, X_val, y_val, X_test, y_test - это ваши данные
spec_height = test_ecg["CWT"][0].shape[1]
spec_width = test_ecg["CWT"][0].shape[2]
channels = 2  # для двух каналов в спектрограмме
conv_shape_1 = 7
conv_shape_2 = 7
opt = SGD(learning_rate = 0.0001)

# Создание модели
model = models.Sequential()
model.add(layers.Conv2D(32, (conv_shape_1, conv_shape_2), activation='relu', input_shape=(spec_height, spec_width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (conv_shape_1, conv_shape_2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (conv_shape_1, conv_shape_2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

# Компиляция модели
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val))

# Оценка производительности на тестовом наборе
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/25
6/6 [==============================] - 30s 5s/step - loss: 0.2173 - accuracy: 0.9056 - val_loss: 0.2061 - val_accuracy: 1.0000
Epoch 2/25
6/6 [==============================] - 29s 5s/step - loss: 0.1731 - accuracy: 0.9444 - val_loss: 0.1581 - val_accuracy: 1.0000
Epoch 3/25
6/6 [==============================] - 29s 5s/step - loss: 0.1468 - accuracy: 0.8889 - val_loss: 0.1235 - val_accuracy: 1.0000
Epoch 4/25
6/6 [==============================] - 29s 5s/step - loss: 0.1148 - accuracy: 0.8667 - val_loss: 0.0855 - val_accuracy: 1.0000
Epoch 5/25
6/6 [==============================] - 29s 5s/step - loss: 0.1773 - accuracy: 0.7944 - val_loss: 0.0379 - val_accuracy: 1.0000
Epoch 6/25
6/6 [==============================] - 30s 5s/step - loss: 0.2894 - accuracy: 0.7833 - val_loss: 0.0209 - val_accuracy: 0.9500
Epoch 7/25
6/6 [==============================] - 30s 5s/step - loss: 8.7168 - accuracy: 0.7111 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 8/25
6/6 [==============

In [14]:
# Предположим, что X_train, y_train, X_val, y_val, X_test, y_test - это ваши данные
spec_height = test_ecg["CWT"][0].shape[1]
spec_width = test_ecg["CWT"][0].shape[2]
channels = 2  # для двух каналов в спектрограмме
conv_shape_1 = 7
conv_shape_2 = 7
opt = SGD(learning_rate = 0.001)

# Создание модели
model = models.Sequential()
model.add(layers.Conv2D(32, (conv_shape_1, conv_shape_2), activation='relu', input_shape=(spec_height, spec_width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (conv_shape_1, conv_shape_2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (conv_shape_1, conv_shape_2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

# Компиляция модели
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val))

# Оценка производительности на тестовом наборе
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)



Epoch 1/25


12/12 [==============================] - 82s 7s/step - loss: 379591.7812 - accuracy: 0.6389 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 2/25
12/12 [==============================] - 78s 7s/step - loss: 0.5439 - accuracy: 0.5194 - val_loss: 0.1406 - val_accuracy: 1.0000
Epoch 3/25
12/12 [==============================] - 86s 7s/step - loss: 0.2412 - accuracy: 0.5056 - val_loss: 0.2294 - val_accuracy: 1.0000
Epoch 4/25
12/12 [==============================] - 80s 7s/step - loss: 0.2954 - accuracy: 0.5500 - val_loss: 0.2063 - val_accuracy: 1.0000
Epoch 5/25
12/12 [==============================] - 81s 7s/step - loss: 0.3001 - accuracy: 0.5028 - val_loss: 0.2236 - val_accuracy: 1.0000
Epoch 6/25
12/12 [==============================] - 88s 7s/step - loss: 0.3347 - accuracy: 0.5167 - val_loss: 0.1919 - val_accuracy: 1.0000
Epoch 7/25
12/12 [==============================] - 83s 7s/step - loss: 0.3672 - accuracy: 0.5083 - val_loss: 0.1257 - val_accuracy: 1.0000
Epoch 8/25
